In [1]:
#!pip install nltk
#!pip install keras
#!pip install tensorflow
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

In [2]:
import json
import random
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
from tensorflow.keras.optimizers import SGD

In [3]:
lemmantizer = WordNetLemmatizer()

In [5]:
intents = json.loads(open('C:/Users/Admin/Python LL Docs/AI Chatbot/intents.json').read())

In [6]:
new_vocab = []
words = []
classes = []
documents = []
ignore_letters = ["?","!",".",",",":"]

In [7]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        words.append(word_list)
        documents.append((word_list,intent["tag"]))
        if intent not in classes:
            classes.append(intent["tag"])

print(words)

[['hello'], ['hey'], ['hi'], ['namaste'], ['yo'], ['Is', 'anyone', 'there', '?'], ['Hola'], ['See', 'ya'], ['bye'], ['get', 'lost'], ['Till', 'next', 'time'], ['bbye'], ['goodbye'], ['catch', 'up', 'later'], ['got', 'ta', 'go'], ['see', 'you', 'later'], ['I', "'m", 'leaving'], ['Have', 'a', 'good', 'day'], ['How', 'are', 'you', '?'], ['How', 'are', 'you', 'doing', '?'], ['What', "'s", 'up', '?'], ['How', "'s", 'it', 'hanging', '?'], ['how', 'is', 'you'], ['are', 'you', 'alright', '?'], ['are', 'you', 'doing', 'good', '?'], ['how', 'old', '?'], ['how', 'old', 'are', 'you', '?'], ['what', 'is', 'your', 'age'], ['age', '?'], ['how', 'long', 'were', 'you', 'created', '?'], ['how', 'long', 'were', 'you', 'born', '?'], ['what', "'s", 'your', 'name'], ['name', '?'], ['how', 'may', 'I', 'address', 'you', '?'], ['your', 'good', 'name', 'please', '?'], ['what', 'are', 'you', 'called', '?'], ['who', 'are', 'you', '?'], ['Thanks'], ['Thank', 'you'], ['That', "'s", 'helpful'], ['Awesome', ',', 'tha

In [8]:
new = words
words = []
for i in new:
    for j in i:
        words.append(j)

In [9]:
words = [lemmantizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [10]:
classes = sorted(set(classes))

In [11]:
pickle.dump(words,open("words.pkl","wb"))
pickle.dump(classes,open("classes.pkl","wb"))

In [12]:
training = []
output_empty = [0]*len(classes)

In [13]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmantizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag,output_row])

In [14]:
random.shuffle(training)
training = np.array(training)

C:\Users\Admin\AppData\Local\Temp\ipykernel_74448\2195589081.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [15]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [16]:
model = Sequential()

In [17]:
model.add(Dense(500,input_shape=(len(train_x[0]),),activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(250,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(len(train_y[0]),activation="softmax"))

In [18]:
sgd = SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=["accuracy"])

C:\Users\Admin\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [19]:
%timeit
hist = model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose = 1)

Epoch 1/200
21/21 [==============================] - 2s 4ms/step - loss: 2.9484 - accuracy: 0.0495
Epoch 2/200
21/21 [==============================] - 0s 4ms/step - loss: 2.8282 - accuracy: 0.2376
Epoch 3/200
21/21 [==============================] - 0s 4ms/step - loss: 2.7003 - accuracy: 0.2178
Epoch 4/200
21/21 [==============================] - 0s 4ms/step - loss: 2.5626 - accuracy: 0.3168
Epoch 5/200
21/21 [==============================] - 0s 3ms/step - loss: 2.3936 - accuracy: 0.3762
Epoch 6/200
21/21 [==============================] - 0s 4ms/step - loss: 2.1829 - accuracy: 0.5446
Epoch 7/200
21/21 [==============================] - 0s 3ms/step - loss: 1.9483 - accuracy: 0.4851
Epoch 8/200
21/21 [==============================] - 0s 4ms/step - loss: 1.7459 - accuracy: 0.5446
Epoch 9/200
21/21 [==============================] - 0s 4ms/step - loss: 1.5434 - accuracy: 0.5644
Epoch 10/200
21/21 [==============================] - 0s 4ms/step - loss: 1.3918 - accuracy: 0.6040
Epoch 11/

21/21 [==============================] - 0s 3ms/step - loss: 0.1076 - accuracy: 0.9307
Epoch 84/200
21/21 [==============================] - 0s 3ms/step - loss: 0.1014 - accuracy: 0.9505
Epoch 85/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0847 - accuracy: 0.9703
Epoch 86/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0982 - accuracy: 0.9406
Epoch 87/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0678 - accuracy: 0.9703
Epoch 88/200
21/21 [==============================] - 0s 3ms/step - loss: 0.1050 - accuracy: 0.9505
Epoch 89/200
21/21 [==============================] - 0s 4ms/step - loss: 0.1031 - accuracy: 0.9406
Epoch 90/200
21/21 [==============================] - 0s 4ms/step - loss: 0.1085 - accuracy: 0.9604
Epoch 91/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0856 - accuracy: 0.9604
Epoch 92/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0967 - accuracy: 0.9406
Epoch 93/200


21/21 [==============================] - 0s 4ms/step - loss: 0.0770 - accuracy: 0.9505
Epoch 165/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0785 - accuracy: 0.9604
Epoch 166/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0722 - accuracy: 0.9604
Epoch 167/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0655 - accuracy: 0.9703
Epoch 168/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0863 - accuracy: 0.9406
Epoch 169/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0765 - accuracy: 0.9604
Epoch 170/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0968 - accuracy: 0.9307
Epoch 171/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0744 - accuracy: 0.9703
Epoch 172/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0987 - accuracy: 0.9505
Epoch 173/200
21/21 [==============================] - 0s 4ms/step - loss: 0.0906 - accuracy: 0.9505
Epoc

In [20]:
model.save("chatbot_model.h5",hist)